<a href="https://colab.research.google.com/github/pedrosousaalves/monitoramento_eletrico_grupo5/blob/main/Monitoramento_Eletrico_Grupo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DADOS BASE

# Lista de Circuitos
circuitos = []

# Dicionário de Limites de Segurança
limites = {
    "iluminacao": {"i_max": 10.0, "fp_min": 0.9, "tensao_nom": 220},
    "motor": {"i_max": 20.0, "fp_min": 0.75, "tensao_nom": 220},
    "tomada": {"i_max": 15.0, "fp_min": 0.8, "tensao_nom": 127},
    "alimentador": {"i_max": 40.0, "fp_min": 0.92, "tensao_nom": 220},
}
tolerancia_tensao = 0.10 # 10%

# Dados de Uso e Tarifa
# O usuário inserirá os valores, mas a estrutura de tarifa é fixa.
TARIFA_RS_KWH = 0.75

In [ ]:
# 2. FUNÇÕES DE LÓGICA

def dentro_da_faixa(circuito):
    """Verifica se um circuito está dentro dos limites de segurança."""
    nome, tipo, v, i, fp, f, data = circuito
    regra = limites.get(tipo)

    if not regra:
        return True

    v_min = regra["tensao_nom"] * (1 - tolerancia_tensao)
    v_max = regra["tensao_nom"] * (1 + tolerancia_tensao)

    if not (v_min <= v <= v_max) or i > regra["i_max"] or fp < regra["fp_min"]:
        return False

    return True

def modulo_consumo(tempo_ligado_horas):
    """Calcula e exibe o consumo de energia (kWh) e o custo para todos os circuitos."""
    if not circuitos:
        print("Atenção: Nenhum circuito cadastrado para calcular o consumo.")
        return

    print("\n RESULTADO DO CÁLCULO DE CONSUMO ")
    print(f"Tempo de Uso Informado: {tempo_ligado_horas:.1f} horas | Tarifa: R$ {TARIFA_RS_KWH:.2f}/kWh")
    print(f"\n{'Circuito':<20}{'P (kW)':>8}{'Consumo (kWh)':>15}{'Custo (R$)':>12}")
    print("-" * 55)

    custo_total = 0.0

    for c in circuitos:
        nome, tipo, v, i, fp, f, data = c

        # P(kW) = (V * I * FP) / 1000
        potencia_kw = (v * i * fp) / 1000.0

        # Consumo (kWh)
        consumo_kwh = potencia_kw * tempo_ligado_horas

        # Custo (R$)
        custo_rs = consumo_kwh * TARIFA_RS_KWH
        custo_total += custo_rs

        print(f"{nome:<20}{potencia_kw:>8.3f}{consumo_kwh:>15.3f}{custo_rs:>12.2f}")

    print("-" * 55)
    print(f"{'Custo Total Diário Estimado:':<38}{custo_total:>16.2f} R$")

def executar_analises():
    """Realiza análises de conformidade e exibe o resumo."""
    if not circuitos:
        print("Atenção: Nenhum circuito cadastrado para análise.")
        return

    print("\n\n===  ANÁLISE DE CONFORMIDADE ===")
    print(f"{'Circuito':<20}{'V (V)':>7}{'I (A)':>7}{'FP':>5}{'Status':>15}")
    print("-" * 54)

    fora_da_faixa = 0

    for c in circuitos:
        nome, _, v, i, fp, _, _ = c
        conforme = dentro_da_faixa(c)
        status = "CONFORME" if conforme else "**NÃO CONFORME**"
        if not conforme:
            fora_da_faixa += 1

        print(f"{nome:<20}{v:>7.0f}{i:>7.2f}{fp:>5.2f}{status:>15}")

    print("\n--- RESUMO GERAL ---")
    print(f"Total de Circuitos Analisados: {len(circuitos)}")
    print(f"Total Fora da Faixa: {fora_da_faixa}")

    # Pergunta o tempo de uso para rodar o módulo de consumo
    try:
        tempo_ligado = float(input("\nDigite o tempo de uso diário médio (em horas) para o cálculo de Consumo: "))
        modulo_consumo(tempo_ligado)
    except ValueError:
        print("Erro: O tempo de uso deve ser um número. O cálculo de consumo foi ignorado.")

In [ ]:
# FUNÇÕES DE ENTRADA DO USUÁRIO

def adicionar_circuito():
    """Permite ao usuário adicionar um novo circuito à lista."""
    print("\n--- NOVO CIRCUITO ---")
    nome = input("Nome do Circuito (Ex: Sala Principal): ").strip()

    tipos_validos = limites.keys()
    while True:
        tipo = input(f"Tipo ({', '.join(tipos_validos)}): ").strip().lower()
        if tipo in tipos_validos:
            break
        print(f"Tipo inválido. Escolha entre: {', '.join(tipos_validos)}")

    try:
        v = float(input("Tensão (V): "))
        i = float(input("Corrente (A): "))
        fp = float(input("Fator de Potência (Ex: 0.95): "))
        f = float(input("Frequência (Hz): "))
        data = input("Data da Medição (Ex: 26/11/2025): ").strip()

        novo_circuito = [nome, tipo, v, i, fp, f, data]
        circuitos.append(novo_circuito)
        print(f"\n Circuito '{nome}' adicionado com sucesso!")
    except ValueError:
        print("\n ERRO: Tensão, Corrente, FP ou Frequência devem ser números.")

def registrar_medicao():
    """Permite ao usuário atualizar medições de um circuito existente."""
    if not circuitos:
        print("Atenção: Não há circuitos cadastrados para registrar medição.")
        return

    print("\n--- REGISTRAR MEDIÇÃO ---")
    nomes = [c[0] for c in circuitos]
    print("Circuitos disponíveis:", ", ".join(nomes))
    nome = input("Digite o nome exato do circuito para atualizar: ").strip()

    for c in circuitos:
        if c[0] == nome:
            print(f"Circuito '{nome}' encontrado. Digite os novos valores (deixe em branco para manter o atual):")

            try:
                v_novo = input(f"Nova Tensão (V, atual {c[2]}): ").strip()
                if v_novo: c[2] = float(v_novo)

                i_novo = input(f"Nova Corrente (A, atual {c[3]}): ").strip()
                if i_novo: c[3] = float(i_novo)

                fp_novo = input(f"Novo Fator de Potência (atual {c[4]}): ").strip()
                if fp_novo: c[4] = float(fp_novo)

                # A frequência (F) é geralmente constante

                print(f"Medição de '{nome}' atualizada com sucesso.")
                return
            except ValueError:
                print("\n ERRO: Valores digitados devem ser números. Nenhuma alteração salva.")
                return

    print(f"Circuito '{nome}' não encontrado.")

In [ ]:
# 4. MENU PRINCIPAL

def main():
    """Controla o fluxo do programa."""
    while True:
        print("\n=============================================")
        print("=== SISTEMA DE MONITORAMENTO ELÉTRICO (v. Simples) ===")
        print("1 - Adicionar Novo Circuito")
        print("2 - Registrar Nova Medição (Atualizar valores)")
        print("3 - EXECUTAR ANÁLISE E CÁLCULO DE CUSTO")
        print("0 - Sair")
        print("=============================================")

        opc = input("Escolha uma opção: ")

        if opc == "1":
            adicionar_circuito()
        elif opc == "2":
            registrar_medicao()
        elif opc == "3":
            executar_analises()
        elif opc == "0":
            print("Sistema encerrado. Bom trabalho!")
            break
        else:
            print("Opção inválida. Tente novamente.")

# Roda o programa
if __name__ == "__main__":
    main()


=== SISTEMA DE MONITORAMENTO ELÉTRICO (v. Simples) ===
1 - Adicionar Novo Circuito
2 - Registrar Nova Medição (Atualizar valores)
3 - ✨ **EXECUTAR ANÁLISE E CÁLCULO DE CUSTO** ✨
0 - Sair
Escolha uma opção: 1

--- NOVO CIRCUITO ---
Nome do Circuito (Ex: Sala Principal): sala principal
